In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import libraries

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from yellowbrick.regressor import PredictionError, ResidualsPlot
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import RadViz
plt.rcParams["figure.figsize"] = (9,5)
import warnings
warnings.filterwarnings('ignore')

### **EDA**

In [1]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [1]:
df.shape

In [1]:
df.info()

In [1]:
df.isnull().sum()

#### EDA - TO DO LIST
1. id column will be dropped
2. each column will maked value_counts 
3. checking null values
4. checking oulier
5. visulization (ex:heatmap)


In [1]:
df = df.drop("id", axis=1)
df.head()

In [1]:
for i in df.select_dtypes('object').columns:
    #if len(df[i].unique())<100:
        print(i , "\n""\n", df[i].value_counts(),"\n","\n","**********************************""\n\n")
    #else :
        #print("\n","***",i,"***","\n")

In [1]:
#### 'other' is dropping from gender column 

df.drop(df[(df["gender"] == "Other")].index, axis = 0, inplace=True)

## second way
## df = df[df["gender"] != "Other"]

In [1]:
df.gender.value_counts() ## we can see the 'other' dropped

In [1]:
for i in df.select_dtypes(['int64','float64']).columns:
    if len(df[i].unique())<100:
        print(i , "\n""\n", df[i].value_counts(dropna = False),"\n","\n","**********************************""\n\n")
    else :
        print("\n","***",i,"***","\n")

In [1]:
sns.pairplot(df, hue='stroke')

In [1]:
sns.heatmap(df.corr(),annot = True)

In [1]:
##df['bmi_cat'] = pd.cut(df['bmi'], bins = [0, 19, 25,30,10000], labels = ['Underweight', 'Ideal', 'Overweight', 'Obesity'])
##df['age_cat'] = pd.cut(df['age'], bins = [0,13,18, 45,60,200], labels = ['Children', 'Teens', 'Adults','Mid Adults','Elderly'])


### bmi column


In [1]:
df.bmi.value_counts(dropna=False)

In [1]:
sns.scatterplot(x='bmi', y='age', data=df)

In [1]:
df.bmi.fillna(df['bmi'].mean(), inplace=True)

In [1]:
df.bmi.isnull().sum()

In [1]:
# pip install pandas-profiling
import pandas_profiling
pandas_profiling.ProfileReport(df)

In [1]:
df_new = pd.get_dummies(df, drop_first = True)
df_new.head()

### Train - Test Split

In [1]:
X = df_new.drop('stroke',axis=1)
y = df_new['stroke']

In [1]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 55)

In [1]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Scaling

In [1]:
scaler = MinMaxScaler()

In [1]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Modelling

In [1]:
from sklearn.linear_model import LogisticRegression

In [1]:
log_model = LogisticRegression(class_weight='balanced')

In [1]:
log_model.fit(X_train_scaled, y_train)

In [1]:
y_pred = log_model.predict(X_test_scaled)

In [1]:
y_pred_proba = log_model.predict_proba(X_test_scaled)

In [1]:
from sklearn.metrics import confusion_matrix, classification_report

In [1]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))